0.39467

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, log_loss
from xgboost import XGBClassifier
import scipy.stats as st

# === Preprocessing Function ===
def preprocess_data(df, is_train=True):
    df = df.copy()

    # Parse datetime columns
    df['Intake Time'] = pd.to_datetime(df.get('Intake Time'), errors='coerce')
    df['Intake Year'] = df['Intake Time'].dt.year
    df['Intake Month'] = df['Intake Time'].dt.month
    df['Intake Day'] = df['Intake Time'].dt.day
    df['Intake Weekday'] = df['Intake Time'].dt.weekday
    df['Intake Hour'] = df['Intake Time'].dt.hour

    if is_train and 'Outcome Time' in df.columns:
        df['Outcome Time'] = pd.to_datetime(df.get('Outcome Time'), errors='coerce')
        df['Length of Stay'] = (df['Outcome Time'] - df['Intake Time']).dt.total_seconds() / (24 * 3600)

    # Function to convert age strings to days
    def parse_age(age_str):
        if pd.isna(age_str):
            return np.nan
        try:
            num = int(''.join(filter(str.isdigit, str(age_str))))
            unit = ''.join(filter(str.isalpha, str(age_str).lower()))
            if 'year' in unit:
                return num * 365
            elif 'month' in unit:
                return num * 30
            elif 'week' in unit:
                return num * 7
            elif 'day' in unit:
                return num
            return np.nan
        except:
            return np.nan

    df['Age in Days'] = df.get('Age upon Intake', '').apply(parse_age)

    if 'Date of Birth' in df.columns:
        df['Date of Birth'] = pd.to_datetime(df.get('Date of Birth'), errors='coerce')
        df['Age from DOB'] = (df['Intake Time'] - df['Date of Birth']).dt.days
        df['Age in Days'] = df['Age in Days'].fillna(df['Age from DOB'])

    df['Is Mix'] = df.get('Breed', '').str.contains('Mix', case=False, na=False).astype(int)
    df['Primary Color'] = df.get('Color', '').str.split('/').str[0]
    df['Secondary Color'] = df.get('Color', '').str.split('/').str[1].fillna('None')
    df['Color Count'] = df.get('Color', '').str.count('/') + 1
    df['Sex'] = df.get('Sex upon Intake', '').str.extract('([A-Za-z]+)', expand=False)
    df['Neutered'] = df.get('Sex upon Intake', '').str.contains('Neutered|Spayed', case=False, na=False).astype(int)
    df['Has Name'] = df.get('Name').notna().astype(int) if 'Name' in df.columns else 0
    df['In Austin'] = df.get('Found Location', '').str.contains('Austin', case=False, na=False).astype(int)

    # Replace rare categories in these columns
    for col in ['Intake Condition', 'Animal Type']:
        if col in df.columns:
            counts = df[col].value_counts()
            rare = counts[counts < 5].index
            df[col] = df[col].replace(rare, 'Other')

    outcome = df['Outcome Type'] if is_train and 'Outcome Type' in df.columns else None

    features = [
        'Intake Type', 'Intake Condition', 'Animal Type', 'Sex', 'Neutered',
        'Age in Days', 'Is Mix', 'Has Name', 'In Austin',
        'Intake Year', 'Intake Month', 'Intake Day', 'Intake Weekday', 'Intake Hour',
        'Primary Color', 'Secondary Color', 'Color Count'
    ]
    if is_train and 'Length of Stay' in df.columns:
        features.append('Length of Stay')

    df = df[[col for col in features if col in df.columns]]
    df.dropna(axis=1, how='all', inplace=True)

    return (df, outcome) if is_train else df

# === Load Data ===
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# === Preprocess Data ===
X, y = preprocess_data(train_df, is_train=True)
X_test = preprocess_data(test_df, is_train=False)

# Remove 'Length of Stay' from training if missing in test set
if 'Length of Stay' in X.columns and 'Length of Stay' not in X_test.columns:
    X = X.drop(columns=['Length of Stay'])
X_test = X_test.loc[:, X.columns]

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Extra safety: drop all-NaN columns
X_train.dropna(axis=1, how='all', inplace=True)
X_val.dropna(axis=1, how='all', inplace=True)
X_test.dropna(axis=1, how='all', inplace=True)

# === Define Feature Lists for Preprocessing ===
numeric_features = [
    'Age in Days', 'Intake Year', 'Intake Month', 'Intake Day',
    'Intake Weekday', 'Intake Hour', 'Color Count'
]
categorical_features = [
    'Intake Type', 'Intake Condition', 'Animal Type', 'Sex',
    'Primary Color', 'Secondary Color'
]
if 'Length of Stay' in X_train.columns:
    numeric_features.append('Length of Stay')

numeric_features = [f for f in numeric_features if f in X_train.columns]
categorical_features = [f for f in categorical_features if f in X_train.columns]

# === Build Preprocessing Pipeline ===
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features)
])

# === Build Combined VotingClassifier Pipeline ===
# This VotingClassifier combines three models: RandomForest, XGBoost, and MLP.
voting_clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(random_state=42, n_jobs=-1)),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, n_jobs=-1)),
            ('mlp', MLPClassifier(random_state=42))
        ],
        voting='soft', n_jobs=-1
    ))
])

# === Define a Reduced Hyperparameter Space for Faster Tuning ===
param_grid = {
    'classifier__rf__n_estimators': [100, 150],
    'classifier__rf__max_depth': [12],
    'classifier__xgb__n_estimators': [100, 150],
    'classifier__xgb__max_depth': [4],
    'classifier__xgb__learning_rate': [0.1],
    'classifier__mlp__hidden_layer_sizes': [(100,), (100, 50)],
    'classifier__mlp__alpha': [0.0001],
    'classifier__mlp__activation': ['relu'],
    'classifier__mlp__max_iter': [300]
}

# Use RandomizedSearchCV for a faster search (20 iterations)
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(voting_clf, param_distributions=param_grid,
                                   n_iter=20, cv=3, scoring='accuracy',
                                   n_jobs=-1, verbose=1, random_state=42)
print("Tuning the Combined VotingClassifier (RandomizedSearchCV)...")
random_search.fit(X_train, y_train)

# === Evaluate on the Validation Set ===
best_model = random_search.best_estimator_
val_pred = best_model.predict(X_val)
val_proba = best_model.predict_proba(X_val)

print("\n--- Validation Results ---")
print("Best Parameters:", random_search.best_params_)
print(f"Validation Accuracy: {accuracy_score(y_val, val_pred):.4f}")
print(f"Validation Log Loss: {log_loss(y_val, val_proba):.4f}")

# === Retrain the Best Model on Full Training Data ===
best_model.fit(X, y_encoded)
test_pred = best_model.predict(X_test)

# === Save Final Submission ===
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'Outcome Type': le.inverse_transform(test_pred)
})
submission.to_csv('submission_combined_optimized.csv', index=False)
print("\n📄 Submission file saved as 'submission_combined_optimized.csv'")
print(submission.head())


<ipython-input-22-33369bf525f4>:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Outcome Time'] = pd.to_datetime(df.get('Outcome Time'), errors='coerce')
<ipython-input-22-33369bf525f4>:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Intake Time'] = pd.to_datetime(df.get('Intake Time'), errors='coerce')
<ipython-input-22-33369bf525f4>:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date of Birth'] = pd.to_datetime(df.get('Date of Birth'), errors='coerce')


Tuning the Combined VotingClassifier (RandomizedSearchCV)...
Fitting 2 folds for each of 5 candidates, totalling 10 fits

--- Validation Results ---
Best Parameters: {'classifier__xgb__n_estimators': 150, 'classifier__xgb__max_depth': 4, 'classifier__xgb__learning_rate': 0.1, 'classifier__rf__n_estimators': 150, 'classifier__rf__max_depth': 12, 'classifier__mlp__max_iter': 300, 'classifier__mlp__hidden_layer_sizes': (100, 50), 'classifier__mlp__alpha': 0.0001, 'classifier__mlp__activation': 'relu'}
Validation Accuracy: 0.6376
Validation Log Loss: 0.8766


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(



📄 Submission file saved as 'submission_optimized.csv'
   Id     Outcome Type
0   1  Return to Owner
1   2         Transfer
2   3  Return to Owner
3   4         Adoption
4   5         Transfer


third- 0.40232

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, log_loss
from xgboost import XGBClassifier
from joblib import Memory
import re

# Set up caching for the preprocessing step
memory = Memory(location='cachedir', verbose=0)

@memory.cache
def preprocess_data(df, is_train=True):
    df = df.copy()

    # Parse datetime columns (vectorized)
    df['Intake Time'] = pd.to_datetime(df.get('Intake Time'), errors='coerce')
    df['Intake Year'] = df['Intake Time'].dt.year
    df['Intake Month'] = df['Intake Time'].dt.month
    df['Intake Day'] = df['Intake Time'].dt.day
    df['Intake Weekday'] = df['Intake Time'].dt.weekday
    df['Intake Hour'] = df['Intake Time'].dt.hour

    if is_train and 'Outcome Time' in df.columns:
        df['Outcome Time'] = pd.to_datetime(df.get('Outcome Time'), errors='coerce')
        df['Length of Stay'] = (df['Outcome Time'] - df['Intake Time']).dt.total_seconds() / (24 * 3600)

    # Vectorized age parsing using regex extraction
    age_extracted = df['Age upon Intake'].astype(str).str.extract(r'(?P<num>\d+)\s*(?P<unit>\w+)', expand=True)
    def convert_age(row):
        try:
            num = int(row['num'])
        except:
            return np.nan
        unit = row['unit'].lower() if pd.notna(row['unit']) else ''
        if 'year' in unit:
            return num * 365
        elif 'month' in unit:
            return num * 30
        elif 'week' in unit:
            return num * 7
        elif 'day' in unit:
            return num
        else:
            return np.nan
    df['Age in Days'] = age_extracted.apply(convert_age, axis=1)

    if 'Date of Birth' in df.columns:
        df['Date of Birth'] = pd.to_datetime(df.get('Date of Birth'), errors='coerce')
        df['Age from DOB'] = (df['Intake Time'] - df['Date of Birth']).dt.days
        df['Age in Days'] = df['Age in Days'].fillna(df['Age from DOB'])

    # Feature engineering for categorical variables
    df['Is Mix'] = df.get('Breed', '').str.contains('Mix', case=False, na=False).astype(int)
    df['Primary Color'] = df.get('Color', '').str.split('/').str[0]
    df['Secondary Color'] = df.get('Color', '').str.split('/').str[1].fillna('None')
    df['Color Count'] = df.get('Color', '').str.count('/') + 1
    df['Sex'] = df.get('Sex upon Intake', '').str.extract(r'([A-Za-z]+)', expand=False)
    df['Neutered'] = df.get('Sex upon Intake', '').str.contains('Neutered|Spayed', case=False, na=False).astype(int)
    df['Has Name'] = df.get('Name').notna().astype(int) if 'Name' in df.columns else 0
    df['In Austin'] = df.get('Found Location', '').str.contains('Austin', case=False, na=False).astype(int)

    # Replace rare categories
    for col in ['Intake Condition', 'Animal Type']:
        if col in df.columns:
            counts = df[col].value_counts()
            rare = counts[counts < 5].index
            df[col] = df[col].replace(rare, 'Other')

    outcome = df['Outcome Type'] if is_train and 'Outcome Type' in df.columns else None

    features = [
        'Intake Type', 'Intake Condition', 'Animal Type', 'Sex', 'Neutered',
        'Age in Days', 'Is Mix', 'Has Name', 'In Austin',
        'Intake Year', 'Intake Month', 'Intake Day', 'Intake Weekday', 'Intake Hour',
        'Primary Color', 'Secondary Color', 'Color Count'
    ]
    if is_train and 'Length of Stay' in df.columns:
        features.append('Length of Stay')

    df = df[[col for col in features if col in df.columns]]
    df.dropna(axis=1, how='all', inplace=True)

    return (df, outcome) if is_train else df

# === Load Data ===
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# === Preprocess (cached) ===
X, y = preprocess_data(train_df, is_train=True)
X_test = preprocess_data(test_df, is_train=False)

# Remove 'Length of Stay' if not available in test set, then align columns
if 'Length of Stay' in X.columns and 'Length of Stay' not in X_test.columns:
    X = X.drop(columns=['Length of Stay'])
X_test = X_test.loc[:, X.columns]

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Drop all-NaN columns (safety check)
X_train.dropna(axis=1, how='all', inplace=True)
X_val.dropna(axis=1, how='all', inplace=True)
X_test.dropna(axis=1, how='all', inplace=True)

# === Define Feature Lists for Preprocessing ===
numeric_features = [
    'Age in Days', 'Intake Year', 'Intake Month', 'Intake Day',
    'Intake Weekday', 'Intake Hour', 'Color Count'
]
categorical_features = [
    'Intake Type', 'Intake Condition', 'Animal Type', 'Sex',
    'Primary Color', 'Secondary Color'
]
if 'Length of Stay' in X_train.columns:
    numeric_features.append('Length of Stay')

numeric_features = [f for f in numeric_features if f in X_train.columns]
categorical_features = [f for f in categorical_features if f in X_train.columns]

# === Build Preprocessing Pipeline ===
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features)
])

# === Build Combined VotingClassifier Pipeline with Weighted Voting ===
voting_clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(random_state=42, n_jobs=-1)),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, n_jobs=-1)),
            ('mlp', MLPClassifier(random_state=42, early_stopping=True))
        ],
        voting='soft',
        weights=[2, 3, 1],  # Adjust weights based on validation results
        n_jobs=-1
    ))
])

# === Define an Expanded but Reasonable Hyperparameter Space ===
param_grid = {
    'classifier__rf__n_estimators': [100, 150, 200],
    'classifier__rf__max_depth': [12, 15],
    'classifier__xgb__n_estimators': [100, 150, 200],
    'classifier__xgb__max_depth': [4, 6],
    'classifier__xgb__learning_rate': [0.1, 0.01],
    'classifier__mlp__hidden_layer_sizes': [(100,), (100, 50), (150, 100)],
    'classifier__mlp__alpha': [0.0001, 0.001],
    'classifier__mlp__activation': ['relu', 'tanh'],
    'classifier__mlp__max_iter': [300]
}

# Use RandomizedSearchCV with n_iter=10 and 3-fold CV for improved tuning
random_search = RandomizedSearchCV(voting_clf, param_distributions=param_grid,
                                   n_iter=10, cv=3, scoring='accuracy',
                                   n_jobs=-1, verbose=1, random_state=42)
print("Tuning the Combined VotingClassifier with Weighted Voting (RandomizedSearchCV)...")
random_search.fit(X_train, y_train)

# === Evaluate on Validation Set ===
best_model = random_search.best_estimator_
val_pred = best_model.predict(X_val)
val_proba = best_model.predict_proba(X_val)
print("\n--- Validation Results ---")
print("Best Parameters:", random_search.best_params_)
print(f"Validation Accuracy: {accuracy_score(y_val, val_pred):.4f}")
print(f"Validation Log Loss: {log_loss(y_val, val_proba):.4f}")

# === Retrain the Best Model on Full Data and Predict on Test Set ===
best_model.fit(X, y_encoded)
test_pred = best_model.predict(X_test)

# === Save Final Submission ===
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'Outcome Type': le.inverse_transform(test_pred)
})
submission.to_csv('third.csv', index=False)
print("\n📄 Submission file saved as 'third.csv'")
print(submission.head())

<ipython-input-2-6121984a2a77>:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Outcome Time'] = pd.to_datetime(df.get('Outcome Time'), errors='coerce')
<ipython-input-2-6121984a2a77>:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Intake Time'] = pd.to_datetime(df.get('Intake Time'), errors='coerce')
<ipython-input-2-6121984a2a77>:55: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date of Birth'] = pd.to_datetime(df.get('Date of Birth'), errors='coerce')


Tuning the Combined VotingClassifier with Weighted Voting (RandomizedSearchCV)...
Fitting 3 folds for each of 10 candidates, totalling 30 fits

--- Validation Results ---
Best Parameters: {'classifier__xgb__n_estimators': 200, 'classifier__xgb__max_depth': 6, 'classifier__xgb__learning_rate': 0.1, 'classifier__rf__n_estimators': 150, 'classifier__rf__max_depth': 15, 'classifier__mlp__max_iter': 300, 'classifier__mlp__hidden_layer_sizes': (100,), 'classifier__mlp__alpha': 0.0001, 'classifier__mlp__activation': 'tanh'}
Validation Accuracy: 0.6453
Validation Log Loss: 0.8568


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(



📄 Submission file saved as 'third.csv'
   Id     Outcome Type
0   1  Return to Owner
1   2         Transfer
2   3  Return to Owner
3   4         Adoption
4   5       Euthanasia


4th-

In [ ]:
!pip install imbalanced-learn xgboost


In [ ]:
import pandas as pd
import numpy as np
import re
import warnings

# Optionally suppress date parsing warnings (informational only)
warnings.filterwarnings("ignore", message="Could not infer format")

# ------------------------------------------------
# 1. Import Required Modules from scikit-learn, imblearn, etc.
# ------------------------------------------------
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, log_loss
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin

from joblib import Memory

# ------------------------------------------------
# 2. Define a Standard Preprocessor for Numeric and Categorical Features
# ------------------------------------------------
# Specify your numeric and categorical feature names (we assume these features are common in your dataset)
numeric_features = ['Age in Days', 'Intake Year', 'Intake Month', 'Intake Day',
                    'Intake Weekday', 'Intake Hour', 'Color Count']
categorical_features = ['Intake Type', 'Intake Condition', 'Animal Type', 'Sex',
                        'Primary Color', 'Secondary Color']

# (Optionally include 'Length of Stay' if present)
def get_feature_lists(X):
    num_feats = list(numeric_features)
    cat_feats = list(categorical_features)
    if 'Length of Stay' in X.columns:
        num_feats.append('Length of Stay')
    num_feats = [f for f in num_feats if f in X.columns]
    cat_feats = [f for f in cat_feats if f in X.columns]
    return num_feats, cat_feats

# Build pipelines for numeric and categorical features
def build_preprocessor(X):
    num_feats, cat_feats = get_feature_lists(X)
    numeric_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    preproc = ColumnTransformer([
        ('num', numeric_pipeline, num_feats),
        ('cat', categorical_pipeline, cat_feats)
    ])
    return preproc

# ------------------------------------------------
# 3. Define a Custom AdvancedPreprocessor Transformer
# ------------------------------------------------
class AdvancedPreprocessor(BaseEstimator, TransformerMixin):
    """
    This transformer applies a preprocessor (one-hot encoding for categoricals
    and scaling for numerics), followed by PCA and then fits KMeans clustering.
    The one-hot encoded cluster labels are appended to the original preprocessed features.
    """
    def __init__(self, n_components=50, n_clusters=5):
        self.n_components = n_components
        self.n_clusters = n_clusters
        self.preprocessor = None  # to be set in fit
        self.pca = PCA(n_components=self.n_components, random_state=42)
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)

    def fit(self, X, y=None):
        self.preprocessor = build_preprocessor(pd.DataFrame(X))  # build preprocessor based on training columns
        X_pre = self.preprocessor.fit_transform(X)
        self.pca.fit(X_pre)
        X_pca = self.pca.transform(X_pre)
        self.kmeans.fit(X_pca)
        return self

    def transform(self, X):
        X_pre = self.preprocessor.transform(X)
        X_pca = self.pca.transform(X_pre)
        clusters = self.kmeans.predict(X_pca)
        # One-hot encode cluster labels using pandas.get_dummies
        cluster_dummies = pd.get_dummies(clusters, prefix="cluster").values
        return np.hstack([X_pre, cluster_dummies])

# ------------------------------------------------
# 4. Preprocessing Function for Data Cleaning (with caching)
# ------------------------------------------------
memory = Memory(location='cachedir', verbose=0)

def preprocess_data(df, is_train=True):
    df = df.copy()

    # Parse datetime columns
    df['Intake Time'] = pd.to_datetime(df.get('Intake Time'), errors='coerce')
    df['Intake Year'] = df['Intake Time'].dt.year
    df['Intake Month'] = df['Intake Time'].dt.month
    df['Intake Day'] = df['Intake Time'].dt.day
    df['Intake Weekday'] = df['Intake Time'].dt.weekday
    df['Intake Hour'] = df['Intake Time'].dt.hour

    if is_train and 'Outcome Time' in df.columns:
        df['Outcome Time'] = pd.to_datetime(df.get('Outcome Time'), errors='coerce')
        df['Length of Stay'] = (df['Outcome Time'] - df['Intake Time']).dt.total_seconds() / (24 * 3600)

    # Parse Age upon Intake using regex extraction (e.g., "2 years")
    age_extracted = df['Age upon Intake'].astype(str).str.extract(r'(?P<num>\d+)\s*(?P<unit>\w+)', expand=True)
    def convert_age(row):
        try:
            num = int(row['num'])
        except:
            return np.nan
        unit = row['unit'].lower() if pd.notna(row['unit']) else ''
        if 'year' in unit:
            return num * 365
        elif 'month' in unit:
            return num * 30
        elif 'week' in unit:
            return num * 7
        elif 'day' in unit:
            return num
        else:
            return np.nan
    df['Age in Days'] = age_extracted.apply(convert_age, axis=1)

    if 'Date of Birth' in df.columns:
        df['Date of Birth'] = pd.to_datetime(df.get('Date of Birth'), errors='coerce')
        df['Age from DOB'] = (df['Intake Time'] - df['Date of Birth']).dt.days
        df['Age in Days'] = df['Age in Days'].fillna(df['Age from DOB'])

    # Feature engineering for categorical/text features
    df['Is Mix'] = df.get('Breed', '').str.contains('Mix', case=False, na=False).astype(int)
    df['Primary Color'] = df.get('Color', '').str.split('/').str[0]
    df['Secondary Color'] = df.get('Color', '').str.split('/').str[1].fillna('None')
    df['Color Count'] = df.get('Color', '').str.count('/') + 1
    df['Sex'] = df.get('Sex upon Intake', '').str.extract(r'([A-Za-z]+)', expand=False)
    df['Neutered'] = df.get('Sex upon Intake', '').str.contains('Neutered|Spayed', case=False, na=False).astype(int)
    df['Has Name'] = df.get('Name').notna().astype(int) if 'Name' in df.columns else 0
    df['In Austin'] = df.get('Found Location', '').str.contains('Austin', case=False, na=False).astype(int)

    for col in ['Intake Condition', 'Animal Type']:
        if col in df.columns:
            counts = df[col].value_counts()
            rare = counts[counts < 5].index
            df[col] = df[col].replace(rare, 'Other')

    outcome = df['Outcome Type'] if is_train and 'Outcome Type' in df.columns else None

    features = [
        'Intake Type', 'Intake Condition', 'Animal Type', 'Sex', 'Neutered',
        'Age in Days', 'Is Mix', 'Has Name', 'In Austin',
        'Intake Year', 'Intake Month', 'Intake Day', 'Intake Weekday', 'Intake Hour',
        'Primary Color', 'Secondary Color', 'Color Count'
    ]
    if is_train and 'Length of Stay' in df.columns:
        features.append('Length of Stay')

    df = df[[col for col in features if col in df.columns]]
    df.dropna(axis=1, how='all', inplace=True)

    return (df, outcome) if is_train else df

preprocess_data = memory.cache(preprocess_data)

# ---------------------------
# 5. Load Data and Preprocess
# ---------------------------
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

X, y = preprocess_data(train_df, is_train=True)
X_test = preprocess_data(test_df, is_train=False)

# Align columns (if "Length of Stay" is not present in test)
if 'Length of Stay' in X.columns and 'Length of Stay' not in X_test.columns:
    X = X.drop(columns=['Length of Stay'])
X_test = X_test.loc[:, X.columns]

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Remove rare classes (if any class appears fewer than 2 times)
class_counts = pd.Series(y_encoded).value_counts()
rare_classes = class_counts[class_counts < 2].index
if len(rare_classes) > 0:
    indices_to_keep = [i for i, label in enumerate(y_encoded) if label not in rare_classes]
    X = X.iloc[indices_to_keep]
    y_encoded = np.array(y_encoded)[indices_to_keep]

# Split into training and validation sets with stratification
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Drop any remaining all-NaN columns (safety check)
X_train.dropna(axis=1, how='all', inplace=True)
X_val.dropna(axis=1, how='all', inplace=True)
X_test.dropna(axis=1, how='all', inplace=True)

# ---------------------------
# 6. Build Preprocessing Pipeline (One-Hot Encoding for Categoricals)
# ---------------------------
num_feats, cat_feats = numeric_features, categorical_features  # From above
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preproc_pipeline = ColumnTransformer([
    ('num', numeric_pipeline, num_feats),
    ('cat', categorical_pipeline, cat_feats)
])

# ---------------------------
# 7. Build AdvancedPreprocessor Transformer (Flattened)
# ---------------------------
class AdvancedPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=50, n_clusters=5):
        self.n_components = n_components
        self.n_clusters = n_clusters
        self.preprocessor = preproc_pipeline  # Use our preprocessor pipeline
        self.pca = PCA(n_components=self.n_components, random_state=42)
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
    def fit(self, X, y=None):
        # Fit preprocessor
        X_pre = self.preprocessor.fit_transform(X)
        self.pca.fit(X_pre)
        X_pca = self.pca.transform(X_pre)
        self.kmeans.fit(X_pca)
        return self
    def transform(self, X):
        X_pre = self.preprocessor.transform(X)
        X_pca = self.pca.transform(X_pre)
        clusters = self.kmeans.predict(X_pca)
        cluster_dummies = pd.get_dummies(clusters, prefix="cluster").values
        return np.hstack([X_pre, cluster_dummies])

advanced_preproc = AdvancedPreprocessor(n_components=50, n_clusters=5)

# ---------------------------
# 8. Build Stacking Classifier as the Ensemble
# ---------------------------
from sklearn.ensemble import StackingClassifier

base_estimators = [
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=12, random_state=42, n_jobs=-1)),
    ('xgb', XGBClassifier(n_estimators=150, max_depth=4, learning_rate=0.1,
                          use_label_encoder=False, eval_metric='mlogloss', random_state=42, n_jobs=-1)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100,50), alpha=0.0001, activation='relu',
                          max_iter=300, random_state=42))
]
meta_learner = LogisticRegression(max_iter=1000, random_state=42)

stacking_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta_learner,
    cv=3,
    n_jobs=-1,
    passthrough=False
)

# ---------------------------
# 9. Build the Combined Pipeline using imblearn's Pipeline (Flattened)
# ---------------------------
combined_pipeline = ImbPipeline(steps=[
    ('advanced_preproc', advanced_preproc),
    ('smote', SMOTE(random_state=42)),
    ('classifier', stacking_clf)
])

# ---------------------------
# 10. Hyperparameter Tuning for the Advanced Pipeline
# ---------------------------
param_grid = {
    'advanced_preproc__n_components': [30, 50, 70],
    'advanced_preproc__n_clusters': [3, 5, 7],
    'classifier__rf__n_estimators': [100, 150, 200],
    'classifier__rf__max_depth': [12, 15],
    'classifier__xgb__n_estimators': [100, 150, 200],
    'classifier__xgb__max_depth': [4, 6],
    'classifier__xgb__learning_rate': [0.1, 0.01],
    'classifier__mlp__hidden_layer_sizes': [(100,), (100,50), (150,100)],
    'classifier__mlp__alpha': [0.0001, 0.001],
    'classifier__mlp__activation': ['relu', 'tanh']
}

random_search = RandomizedSearchCV(combined_pipeline, param_distributions=param_grid,
                                   n_iter=10, cv=3, scoring='accuracy',
                                   n_jobs=-1, verbose=1, random_state=42)
print("Tuning the Advanced Pipeline (PCA + K-Means + SMOTE + Stacking)...")
random_search.fit(X_train, y_train)

# ---------------------------
# 11. Evaluate on the Validation Set
# ---------------------------
best_model = random_search.best_estimator_
val_pred = best_model.predict(X_val)
val_proba = best_model.predict_proba(X_val)
print("\n--- Validation Results ---")
print("Best Parameters:", random_search.best_params_)
print(f"Validation Accuracy: {accuracy_score(y_val, val_pred):.4f}")
print(f"Validation Log Loss: {log_loss(y_val, val_proba):.4f}")

# ---------------------------
# 12. Retrain Best Model on Full Data and Predict on Test Set
# ---------------------------
best_model.fit(X, y_encoded)
test_pred = best_model.predict(X_test)

# ---------------------------
# 13. Save Final Submission
# ---------------------------
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'Outcome Type': le.inverse_transform(test_pred)
})
submission.to_csv('4g.csv', index=False)
print("\n📄 Submission file saved as '4.csv'")
print(submission.head())


Tuning the Advanced Pipeline (PCA + K-Means + SMOTE + Stacking)...
Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/imblearn/pipeline.py", line 518, in fit
    Xt, yt = self._fit(X, y, routed_params, raw_params=params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/imblearn/pipeline.py", line 43